# Alzeimer's Disease Clinical Trials

## Import Library 

In [4]:
## Import Library 
import json
import pandas as pd
import os
import streamlit as st
import plotly.express as px
import seaborn as sns

## Data scripting from Clinical trial.gov

In [2]:
# Initialize a list to collect data
data_list = []

In [4]:
#Loop through each JSON file in the current directory
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".json"):
        file_path = os.path.join(os.getcwd(), filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id_module = data['protocolSection']['identificationModule']
            desc_module = data['protocolSection']['descriptionModule']
            contacts_locations_module = data['protocolSection'].get('contactsLocationsModule', {})
            sponsor_module = data['protocolSection'].get('sponsorCollaboratorsModule', {})
            responsible_party = sponsor_module.get('responsibleParty', {})
            additional_pi_section = data.get('additionalPISection', [{}])

            # Extract required data from JSON file
            nct_id = id_module.get("nctId", "")
            detailed_description = desc_module.get("detailedDescription", "")

            # Extracting PI information from various sections
            overall_officials = contacts_locations_module.get("overallOfficials", [])
            if overall_officials:
                pi_info = overall_officials[0]
                pi = pi_info.get("name", "")
                role = pi_info.get("role", "")
                affiliation = pi_info.get("affiliation", "")
            elif responsible_party:
                pi = responsible_party.get("investigatorFullName", "")
                role = responsible_party.get("investigatorTitle", "")
                affiliation = responsible_party.get("investigatorAffiliation", "")
            elif additional_pi_section:
                pi_info = additional_pi_section[0]
                pi = pi_info.get("name", "")
                affiliation = pi_info.get("affiliation", "")
                role = ""
# Append the processed data to the list
            data_list.append({
                "nct_id": nct_id,
                "detailed_description": detailed_description,
                "pi": pi,
                "role": role,
                "affiliation": affiliation
            })

            # Convert the list of dictionaries to a DataFrame
            df_json = pd.DataFrame(data_list)

            csv_file = "ctg-studies.csv"
            df_csv = pd.read_csv(csv_file)
            df_csv.rename(columns={'NCT Number': 'nct_id'}, inplace=True)

            # Merge the DataFrames
            merged_df = pd.merge(df_json, df_csv, on='nct_id', how='left')

            # Save the merged DataFrame to an Excel file
            merged_df.to_excel("merged_output.xlsx", index=False)

## Data processing 

In [ ]:
file_path = 'merged_output.xlsx'
df_merged = pd.read_excel(file_path)

### Data Cleaning 

In [ ]:
cleaned_df = df_merged.dropna(subset=['Phases','Sponsor','Start Date','affiliation', 'pi'])
cleaned_df = cleaned_df[(cleaned_df['Study Status']!= 'WITHDRAWN')]
# Convert the 'Start Date' column to datetime format
cleaned_df['Start Date'] = pd.to_datetime(cleaned_df['Start Date'], errors='coerce')
cleaned_df['Start Year'] = cleaned_df['Start Date'].dt.year
# Convert the 'Primary Completion Date' column to datetime format
cleaned_df['Primary Completion Date'] = pd.to_datetime(cleaned_df['Start Date'], errors='coerce')
cleaned_df['Completion Year'] = cleaned_df['Primary Completion Date'].dt.year

### Dashboard

In [ ]:
import pandas as pd
import streamlit as st
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

file_path = 'merged_output.xlsx'
df_merged = pd.read_excel(file_path)

cleaned_df = df_merged.dropna(subset=['Phases','Sponsor','Start Date','affiliation', 'pi'])
cleaned_df = cleaned_df[(cleaned_df['Study Status']!= 'WITHDRAWN')]
# Convert the 'Start Date' column to datetime format
cleaned_df['Start Date'] = pd.to_datetime(cleaned_df['Start Date'], errors='coerce')
cleaned_df['Start Year'] = cleaned_df['Start Date'].dt.year
# Convert the 'Primary Completion Date' column to datetime format
cleaned_df['Primary Completion Date'] = pd.to_datetime(cleaned_df['Start Date'], errors='coerce')
cleaned_df['Completion Year'] = cleaned_df['Primary Completion Date'].dt.year

# Page config
st.set_page_config(
    page_title="AD clinical Trial Dashboard",
    page_icon= ":bar_chart",
    layout="wide",
)
# Title of the dashboard
st.title("Alzheimer's Diasease Clinical Trial Dashboard")
st.markdown('## Insight into Industry-Sponsored Alzheimers Disease Trials')

# Slidebar for filtering Start Year
st.sidebar.title("Filter by Year Range")
year_range = st.slider(
    "Select Year Range",
    min_value= int(cleaned_df['Start Year'].min()),
    max_value= int(cleaned_df['Start Year'].max()),
    value=(2010, 2030)
)
# Filter the for study start year
df_start = cleaned_df[(cleaned_df['Start Year'] >= year_range[0]) & (cleaned_df['Start Year'] <= year_range[1])]
grouped_start = df_start.groupby(['Start Year', 'Study Status']).size().reset_index(name='Count')

st.subheader('Registered Clinical Trials by Year Started and Study Status')
fig0, ax0 = plt.subplots(figsize=(10,6))
sns.barplot(x='Start Year', y="Count", hue= 'Study Status', 
            data=grouped_start, palette="Set2", ax=ax0)
ax0.set_xlabel('Year Started')
ax0.set_ylabel('Number of Trials')
ax0.set_title('Registered Trials by Year Started and Study Status')
ax0.set_xticklabels(ax0.get_xticklabels(), rotation=45)
ax0.legend(title='Study Status')
# Display the plot in Streamlit
st.pyplot(fig0)


# Streamlit slider for selecting the year range
st.subheader("Select Year Range for Completed Trials")
start_year, end_year = st.slider(
    "Select the range of years", 
    min_value=2010, max_value=2030, 
    value=(2010, 2030)  
)

# Filter the data based on the selected year range
df_complete = cleaned_df[(cleaned_df['Completion Year'] >= start_year) & 
                            (cleaned_df['Completion Year'] <= end_year)]

# Count the number of trials in the Selected Year Range
num_trials_complete = df_complete.shape[0]
# Streamlit section to show the number of trials in the Selected Year Range
st.subheader("Number of Trials complete in the Selected Year Range")
st.metric(label="Trials complete (Selected Year Range)", value=num_trials_complete)


# Plot 1: Pie Chart for Phases will complete in the Selected Year Range
st.subheader("Phases Distribution of Trials complete in the Selected Year Range")
phase_counts = df_complete['Phases'].value_counts()
fig1, ax1 = plt.subplots()
ax1.pie(phase_counts, labels=phase_counts.index, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')
st.pyplot(fig1)


# Plot 2: Bar Chart for Sponsor vs. Phases (complete in the Selected Year Range)
st.subheader("Sponsor vs. Phases of Trials Complete in the Selected Year Range")
sponsor_phase_counts = df_complete.groupby(['Sponsor', 'Phases']).size().reset_index(name='Counts')
fig2 = px.bar(sponsor_phase_counts, x='Sponsor', y='Counts', color='Phases', barmode='group',
            title="Number of Studies per Sponsor by Phases")
st.plotly_chart(fig2)

# Plot3: Bar Chart for Conditions 
st.subheader("Trials by Condition (complete in the Selected Year Range)")
condition_counts = df_complete.groupby('Conditions').size().reset_index(name='Counts')
fig3 = px.bar(condition_counts, x='Conditions', y='Counts',
            title="Number of Studies per Condition")
st.plotly_chart(fig3)


# New Plot 4: Number of Studies Expected to Complete in the Selected Year Range
st.subheader("Number of Studies Expected to Complete in the Selected Year Range")

# Group by year and sponsor
date_grouped_df = df_complete.groupby(['Completion Year', 'Sponsor']).size().reset_index(name='Count')

# Create a bar chart using Plotly
fig4 = px.bar(date_grouped_df, 
              x='Completion Year', 
              y='Count', 
              color='Sponsor', 
              barmode='group',
              title="Studies Expected to Complete in the Selected Year Range by Sponsor"
              )

st.plotly_chart(fig4)

# Footer
st.write("### Data Source: https://clinicaltrials.gov")